# 🦌 Compounds 🧪


```python

```

In [1]:
import importnb

# import ipyelk.diagram.elk_export
import ipywidgets as W
import networkx as nx
from IPython.display import SVG, JSON, display

import ipyelk
import ipyelk.nx
import ipyelk.tools
import ipyelk.tools.tools

# from ipyelk.contrib.library import logic_gates as logic
from ipyelk import Elk
from ipyelk.contrib.shapes import connectors as conn
from ipyelk.contrib.shapes import shapes
from ipyelk.diagram import elk_export, elk_model
from ipyelk.diagram import layout_options as opt
from ipyelk.diagram import symbol
from ipyelk.diagram.symbol import Symbol
from ipyelk.diagram.defs import ConnectorDef, Def
from ipyelk.diagram.layout_options.layout import ELKRectanglePacking

from typing import Tuple, Union, ClassVar, List, Optional, Dict
from dataclasses import field, dataclass

from uuid import uuid4

from collections import defaultdict
from inspect import isclass
from ipyelk.transform import merge
from ipyelk.diagram.elk_model import strip_none

from ipyelk.contrib.elements import Registry, Node, Port, Compound , Label
from ipyelk.contrib.library import logic_gates

### TODO
* how transformer deals with layoutOptions / properties / cssClasses
    * probably not great to have them in too many places...
    * "inherited" from higher nodes in the graph vs explicitly set in the node data
* label shapes
    * icon view
* sizing for record compounds
* composing compounds

In [2]:
with Registry():
    n1 = Node(
        ports={"a":Port()},
        labels=[Label(text="hello")]
    )
    n1.a._parent
    n1.add_child(Node())
    
    data = n1.to_json()
    
JSON(data)

<IPython.core.display.JSON object>

In [3]:
n1 = Node(labels=[Label(text="hello", labels=[Label(text="subtext")])])
# n1.a = Port()

n1.add_child(Node())
n2 = Node()
n2.a = Port()
ilk = Compound()
ilk.connect(n2.a, n1)


Connection(source=Port(shape=None, labels=[], properties={}, layoutOptions={}, _parent=Node(shape=None, labels=[], properties={}, layoutOptions={}, ports={'a': ...}, children=[])), target=Node(shape=None, labels=[Label(shape=None, labels=[Label(shape=None, labels=[], properties={}, layoutOptions={}, text='subtext')], properties={}, layoutOptions={}, text='hello')], properties={}, layoutOptions={}, ports={}, children=[Node(shape=None, labels=[], properties={}, layoutOptions={}, ports={}, children=[])]))

In [4]:
with Registry():
    display(JSON(n1.to_json()))

<IPython.core.display.JSON object>

In [5]:
# configure app
app = Elk(
    transformer=ipyelk.nx.XELK(
#         layouts={
#             elk_model.ElkRoot: {
#                 "parents": diagram_opts,
#             },
#         },
        source=ilk.source(),
    ),
    layout={"height": "100%"},
    style={
        " .hidden": {
            #             "display": "none",
        }
    },
)
app

Elk(children=[HTML(value='<style>.styled-widget-140342780585872 .hidden{}</style>', layout=Layout(display='Non…

In [7]:
ng = Node(
        shape= logic_gates.Nand_Gate()
    )
n1.add_child(
    ng
)

app.diagram.defs = {
    logic_gates.Nand_Gate.identifier:logic_gates.Nand_Gate.shape
}
app.transformer.source = ilk.source()
from ipyelk.contrib.elements.compond import nx_wrap
nx_ng = nx_wrap(ng, ilk.registry)

In [10]:
ng.shape = logic_gates.Nand_Gate()
app.transformer.source = ilk.source()

In [8]:
app.transformer.get_properties(nx_ng)

{'type': 'node:use', 'shape': {'use': 'nand_gate'}}

In [12]:
JSON(app.transformer.value)

<IPython.core.display.JSON object>

In [14]:
from IPython.display import JSON
JSON([d for *_, d in app.transformer.source[0].edges(data=True)])

<IPython.core.display.JSON object>

In [10]:
JSON(app.transformer.value)

<IPython.core.display.JSON object>

In [11]:
JSON([d for _, d in ilk.source()[0].nodes(data=True)])

<IPython.core.display.JSON object>

In [12]:
logic_gates.Nand_Gate.shape

Def(width=None, height=None, children=[SVG(width=None, height=None, children=[], x=-219.5, y=-55, value='<path xmlns="http://www.w3.org/2000/svg" d="M\n                219.98902,54.008713 L 219.98902,80.967304 C 266.13267,83.386665\n                267.20732,51.416541 219.98902,54.008713 z "/>'), SVG(width=None, height=None, children=[], x=-209, y=-221.5, value='<path xmlns="http://www.w3.org/2000/svg" d="M 254.10886\n                234.75238 A 4.7229962 4.7229962 0 1 1  244.66287,234.75238 A\n                4.7229962 4.7229962 0 1 1  254.10886 234.75238 z"/>')], x=None, y=None)

In [15]:
g, tree = ilk.source(context=Registry())

0b88f331-9243-4dab-b345-23eae26f6378.7a2685cc-eb09-474e-8d59-eca742601189


In [313]:
p = app.selected[0]
p.elkport

ElkPort(id="Node(labels=[], properties={}, layoutOptions={}, ports={'a': Port(labels=[], properties={}, layoutOptions={}, _parent=...)}, children=[]).77f46c2e-36b7-4b9c-8e61-58fcf23c2012", labels=None, layoutOptions=None, height=5, width=5, x=None, y=None, properties=None)

In [286]:
await app.transformer._refresh()

UnboundLocalError: local variable 'elkport' referenced before assignment

In [282]:
[d for d in ilk.source()[0].nodes()]

[Node(labels=[], properties={}, layoutOptions={}, ports={'a': Port(labels=[], properties={}, layoutOptions={}, _parent=...)}, children=[]),
 Node(labels=[Label(labels=[Label(labels=[], properties={}, layoutOptions={}, text='subtext')], properties={}, layoutOptions={}, text='hello')], properties={}, layoutOptions={}, ports={}, children=[Node(labels=[], properties={}, layoutOptions={}, ports={}, children=[])]),
 Node(labels=[], properties={}, layoutOptions={}, ports={}, children=[])]

In [177]:
from IPython.display import JSON
JSON([d for _, d in ilk.source()[0].nodes(data=True)])

<IPython.core.display.JSON object>

In [178]:
from IPython.display import JSON
JSON([d for *_, d in ilk.source()[0].edges(data=True)])

<IPython.core.display.JSON object>

In [157]:
await app.transformer._refresh()

AttributeError: 'Port' object has no attribute 'get'

In [161]:
%debug

> /home/dfreeman6/Documents/ipyelk/py_src/ipyelk/nx/transformer.py(263)get_properties()
    261                     properties += [g_props]
    262         elif hasattr(element, "data"):
--> 263             properties += [element.data.get("properties", {})]
    264         elif isinstance(element, dict):
    265             properties += [element.get("properties", {})]



ipdb>  u


> /home/dfreeman6/Documents/ipyelk/py_src/ipyelk/nx/transformer.py(202)make_elknode()
    200         node_ports = await self.collect_ports(node)
    201 
--> 202         properties = self.get_properties(node, self.get_css(node, ElkNode)) or None
    203 
    204         elk_node = ElkNode(



ipdb>  node


Node(labels=[], _ports={'a': Port(labels=[], _parent=...), 'data': Port(labels=[], _parent=...)})


...

ipdb>  d


> /home/dfreeman6/Documents/ipyelk/py_src/ipyelk/nx/transformer.py(140)transform()
    138         ports: PortMap = {}
    139         for node in visible_nodes:
--> 140             elknode, node_ports = await self.make_elknode(node)
    141             for key, value in node_ports.items():
    142                 ports[key] = value



ipdb>  ll


    121     async def transform(self) -> ElkNode:
    122         """Generate ELK dictionary structure
    123         :return: Root Elk node
    124         :rtype: ElkNode
    125         """
    126         # TODO decide behavior for nodes that exist in the tree but not g
    127         g, tree = self.source
    128         self.clear_registry()
    129         visible_edges, hidden_edges = self.collect_edges()
    130 
    131         # Process visible networkx nodes into elknodes
    132         visible_nodes = [
    133             n for n in g.nodes() if not is_hidden(tree, n, self.HIDDEN_ATTR)
    134         ]
    135 
    136         # make elknodes then connect their hierarchy
    137         elknodes: NodeMap = {}
    138         ports: PortMap = {}
    139         for node in visible_nodes:
--> 140             elknode, node_ports = await self.make_elknode(node)
    141             for key, value in node_ports.items():
    142                 ports[key] = value
    143    

ipdb>  c


In [132]:
with ilk.registry:
    print(n1.to_json())

{'id': '9320643c-9795-499e-ade6-1c67e327c1f7', 'children': [], 'properties': {'type': 'node'}, 'labels': [], 'layoutOptions': {}, 'ports': [], 'width': 0, 'height': 0}


In [122]:
with reg:
    print(list(ilk.edges)[0].to_json())

{'id': 'b519b08e-8ad6-4371-bb19-b6d0ac01d349', 'sources': ['133ec525-c823-47f6-abba-d22a04802de5'], 'targets': ['77fdb214-64fc-4c1d-b143-ba8eaa28d8df'], 'properties': {}, 'layoutOptions': {}, 'labels': []}


In [11]:


class Compound(symbol.Symbol):
    
    ids = {}
    
    def __init__(self):
        
    
    def ids(self):
        for node in self.nodes:
            
    def source(self)->Tuple[nx.MultiDiGraph, nx.DiGraph]:
        id_map = dict()
        g = nx.MultiDiGraph()
        tree = nx.DiGraph()
        # needs to maintain mapping of ids to children nodes
        return (g, tree)
    
    


IndentationError: expected an indented block (<ipython-input-11-a99f2461d8f3>, line 14)

# Experiments for Record Nodes

In [ ]:
app = Elk(
    transformer=ipyelk.nx.XELK(
        #         layouts={
        #             elk_model.ElkRoot: {
        #                 "parents": diagram_opts,
        #             },
        #         },
        #         source=(g, tree),
    ),
    layout={"height": "100%"},
    style={
        " .hidden": {
            #             "display": "none",
        }
    },
)
app

In [ ]:
from dataclasses import dataclass
from typing import Dict


@dataclass
class Record(symbol.Symbol):
    def get_layoutOptions(self) -> Dict:
        return opt.OptionsWidget(
            options=[
                opt.HierarchyHandling(),
                opt.Padding(left=0, right=0, bottom=0, top=0),
                opt.NodeSpacing(spacing=0),
                opt.EdgeNodeSpacing(spacing=0),
                opt.AspectRatio(),
                opt.ExpandNodes(activate=True),
                opt.NodeLabelPlacement(horizontal="center", vertical="center"),
                #         opt.NodeSizeConstraints(),
                opt.ComponentsSpacing(spacing=0),
                opt.NodeSpacing(spacing=0),
            ]
        ).value


# .to_json()
g = nx.Graph()
tree = nx.DiGraph()
r = Record()
g.add_node("a", **r.to_json(id="a"))

In [ ]:
g = nx.Graph()
tree = nx.DiGraph()

diagram_opts = opt.OptionsWidget(
    options=[
        opt.LayoutAlgorithm(),
        opt.HierarchyHandling(),
        #         opt.NodeSpacing(spacing=0),
    ]
).value
record_layout = opt.OptionsWidget(
    options=[
        #         opt.LayoutAlgorithm(value=ELKRectanglePacking.identifier),
        opt.HierarchyHandling(),
        opt.Padding(left=0, right=0, bottom=0, top=0),
        opt.NodeSpacing(spacing=0),
        opt.EdgeNodeSpacing(spacing=0),
        opt.AspectRatio(),
        opt.ExpandNodes(activate=True),
        opt.NodeLabelPlacement(horizontal="center", vertical="center"),
        #         opt.NodeSizeConstraints(),
        opt.ComponentsSpacing(spacing=0),
        opt.NodeSpacing(spacing=0),
    ]
).value

compartment_opts = opt.OptionsWidget(
    options=[
        opt.NodeLabelPlacement(horizontal="center", vertical="center"),
        #         opt.NodeSizeConstraints(),
        opt.NodeSpacing(spacing=0),
    ]
).value

g.add_node("record", labels=[], layoutOptions=record_layout)

compartments = ["a", "b"]
for i, c in enumerate(compartments):
    g.add_node(
        c, width=40, height=36, layoutOptions=compartment_opts, ports=[c + str(i)]
    )
    tree.add_edge("record", c)

# g.add_node("n1")

# configure app
app = Elk(
    transformer=ipyelk.nx.XELK(
        layouts={
            elk_model.ElkRoot: {
                "parents": diagram_opts,
            },
        },
        source=(g, tree),
    ),
    layout={"height": "100%"},
    style={
        " .hidden": {
            #             "display": "none",
        }
    },
)
app

In [ ]:
app.diagram.mark_layout

In [ ]:
g = nx.Graph()
tree = nx.DiGraph()

record_layout = opt.OptionsWidget(
    options=[
        opt.LayoutAlgorithm(value=ELKRectanglePacking.identifier),
        opt.HierarchyHandling(),
        opt.Padding(left=0, right=0, bottom=0, top=0),
        opt.NodeSpacing(spacing=0),
        opt.EdgeNodeSpacing(spacing=0),
        opt.AspectRatio(),
        opt.ExpandNodes(activate=True),
    ]
).value

compartment_opts = opt.OptionsWidget(
    options=[
        opt.NodeLabelPlacement(horizontal="center", vertical="center"),
        opt.NodeSizeConstraints(),
    ]
).value

compartments = ["a", "b"]
for i, c in enumerate(compartments):
    g.add_node(
        c,
        width=40 + 2 * i,
        height=30,
        layoutOptions=compartment_opts,
        #         ports=[c + str(i)]
    )


# configure app
app = Elk(
    transformer=ipyelk.nx.XELK(
        layouts={
            elk_model.ElkRoot: {
                "parents": record_layout,
            },
        },
        source=(g, tree),
    ),
    layout={"height": "100%"},
    style={
        " .hidden": {
            #             "display": "none",
        }
    },
)
app